In [1]:
### REQUIREMENTS
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

!pip install --ignore-installed --index-url https://test.pypi.org/simple/ --no-deps sddk ### our own package under construction, always install to have up-to-date version
import sddk

Looking in indexes: https://test.pypi.org/simple/
  Using cached https://test-files.pythonhosted.org/packages/65/8b/d682c15a7335215ac119538ad8455b408cd7e8be4f6614678888dd2c88ed/sddk-0.0.7-py3-none-any.whl


## configure session and groupurl

In [3]:
### configure session and groupurl
### in the case of "SDAM_root", the group owner is Vojtech with username 648597@au.dk
s, sddk_url = sddk.configure_session_and_url("SDAM_root")

sciencedata.dk username (format '123456@au.dk'): kase@zcu.cz
sciencedata.dk password: ··········
personal connection established
"SDAM_root" owner's username: 648597@au.dk
group connection established with you as member
endpoint for requests has been configured to: https://sciencedata.dk/sharingin/648597@au.dk/SDAM_root/


# Upload the main preprocessed and enriched dataframe from sciencedata.dk


In [10]:
### now we can upload the data from sciencedata into Pandas dataframe
### look at Pandas documention to learn how to navigate Pandas dataframe with their endless functionality
EDH_df = pd.DataFrame(s.get(sddk_url + "SDAM_data/EDH/EDH_inscriptions_rich.json").json())
EDH_df.set_index("id", inplace=True) ### perhaps the best index is the "ID"
EDH_df.head(5) ### use ".head(5)" to inspect first 5 rows of the dataframe

,people,work_status,findspot_modern,last_update,responsible_individual,width,language,literature,height,diplomatic_text,not_before,depth,material,trismegistos_uri,transcription,commentary,edh_geography_uri,country,uri,province_label,modern_region,type_of_monument,present_location,findspot_ancient,not_after,type_of_inscription,letter_size,social_economic_legal_history,findspot,year_of_find,geography,religion,fotos,military,external_image_uris,coordinates,text_cleaned,origdate_text,objecttype
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HD001917,"[{'name': 'L. Ponponius(!) Rufus', 'age: years...",checked with photo,Roma,2014-10-10,Cowey,19 cm,Greek-Latin,"CIG 6916.; AE 1984, 0109. (B); P. Lombardi, Ti...",45 cm,L PONPONIVS RVFVS / VIXIT ANOS XXVII / EIA PON...,0101,5.4 cm,marble: rocks - metamorphic rocks,https://www.trismegistos.org/text/177036,L(ucius) Ponponius(!) Rufus / vixit an(n)os XX...,Wiederverwendung der Tafel als TÃ¼rpfosten. D...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Italy,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,Lazio,tabula,"Roma, Mus. Naz. Rom.","Kephallenia, aus",0200,epitaph,1-2 cm,None,None,None,None,None,None,None,None,"41.8917375,12.4861685",Lucius Ponponius sic Rufus vixit annos XXVII E...,101 AD – 200 AD,"[Tafel, 257]"
HD002097,"[{'gender': 'male', 'cognomen': 'ÎÎ±Î»Î»ÎµÎ½Ï...",checked with photo,"Patrasso - AthÃ­nai, zwischen",2012-03-15,GrÃ¤f,30 cm,Greek-Latin,"CIL 03, 00572.; CIL 03, 07306.; IG 02 (2. Aufl...",146 cm,[ ]ΥΤΟΚΡΑΤΟΡΙ / [ ]ΑΙΣΑΡΙ / [[[ ]]] / [ ]ΥΣΕΒΕ...,0395,None,"Marmor, geÃ¤dert / farbig",https://www.trismegistos.org/text/177037,[Α]ὐτοκράτορι / [Κ]αίσαρι / [[[---]]] / [Ε]ὐσε...,Meilenstein mit zwei griechischen Inschriften...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,mile-/leaguestone,"AthÃ­nai, Epigr. Mus.","Athenae, bei",0397,mile-/leaguestone,2.7 cm,data available,"Dafni, byzantinisches Kloster, bei, sekundÃ¤r ...",None,None,None,None,None,None,"38.012978,23.635883",Αὐτοκράτορι Καίσαρι [] Εὐσεβεῖ Εὐτυχει Σεβαστῶ...,395 AD – 397 AD,"[Meilen-/Leugenstein, 89]"
HD002919,None,no image,AthÃ­nai,2011-04-04,Cowey,(17) cm,Latin,"CIL 03, 06101.; M. Å aÅ¡el Kos, Inscriptiones ...",(15) cm,]S HOSTIVM DEPRESSE[ ] / [ ] CXIIX BELLO MARIT...,-0038,12.5 cm,None,https://www.trismegistos.org/text/177038,------ nave]s hostium depresse[rit ---] / [---...,Es handelt sich um ein Elogium fÃ¼r Agrippa. ...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,base,"AthÃ­nai, Epigr. Mus.",Athenae,-0012,elogium,6.5 cm,None,"Roma-Augustus Tempel, Akropolis",1866,None,None,None,None,None,"37.983175,23.716647",naves hostium depresserit CXIIX bello maritu...,38 BC – 12 BC,"[Basis, 53]"
HD002922,"[{'cognomen': 'Traianus Hadrianus', 'gender': ...",checked with photo,AthÃ­nai,2009-11-17,Cowey,76 cm,Greek-Latin,"CIL 03, 00548. (B); CIL 03, 07281.; PIR (2. Au...",112 cm,[ ]MP CAES DIVI TRAIANI PAR / THICI FIL DIVI N...,0132,48 cm,None,https://www.trismegistos.org/text/177039,[I]mp(eratori) Caes(ari) divi Traiani Par/thic...,(B): Am Anfang von Z. 2 fehlt das TI von nepoti.,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,statue base,"AthÃ­nai, Epigr. Mus.",Athenae,None,honorific inscription,None,None,None,None,data available,None,None,None,None,"37.983175,23.716647",Imperatori Caesari divi Traiani Parthici filio...,132 AD,"[Statuenbasis, 57]"
HD002925,"[{'gender': 'male', 'cognomen': 'Traianus+ Had...",no image,AthÃ­nai,2011-04-04,Cowey,(41) cm,Latin,"CIL 03, 06102.; CIL 03, 07283.; AE 1984, 0822....",(20) cm,[ ] / [ ] / [ ]D[ ] / [ ]R P XVI COS III P P [...,0132,(15) cm,marble: rocks - metamorphic rocks,https://www.trismegistos.org/text/177040,[Imp(eratori) Caesari divi Traiani] / [Parthic...,Rekonstruktion des Inschriftentextes nach CIL...,https://edh-www.adw.uni-heidelberg.d

In [11]:
### how many rows and columns we have
EDH_df.shape

(71260, 39)

In [13]:
### inspect all unique values within "type_of_inscription"
EDH_df["type_of_inscription"].unique()

array(['epitaph', 'mile-/leaguestone', 'elogium', 'honorific inscription',
       'honorific inscription?', 'votive inscription',
       'public legal inscription', None,
       'building/dedicatory inscription?', 'list',
       'public legal inscription?', 'votive inscription?',
       'building/dedicatory inscription', 'identification inscription',
       'prayer', 'defixio', 'private legal inscription', 'acclamation',
       'military diploma', 'owner/artist inscription',
       'identification inscription?', 'owner/artist inscription?',
       'list?', 'epitaph?', 'mile-/leaguestone?', 'calendar?', 'label',
       'boundary inscription?', 'boundary inscription', 'acclamation?',
       'label?', 'seat inscription', 'defixio?',
       'private legal inscription?', 'seat inscription?', 'letter',
       'adnuntiatio', 'prayer?', 'calendar', 'letter?',
       'assignation inscription', 'elogium?', 'assignation inscription?',
       'military diploma?'], dtype=object)

In [34]:
### to get a smaller dataset 
EDH_miles = EDH_df[EDH_df["type_of_inscription"].str.startswith("mile-/lea", na=False)]
len(EDH_miles) ### how long it is?

1524

In [35]:
EDH_miles.head(20)

,people,work_status,findspot_modern,last_update,responsible_individual,width,language,literature,height,diplomatic_text,not_before,depth,material,trismegistos_uri,transcription,commentary,edh_geography_uri,country,uri,province_label,modern_region,type_of_monument,present_location,findspot_ancient,not_after,type_of_inscription,letter_size,social_economic_legal_history,findspot,year_of_find,geography,religion,fotos,military,external_image_uris,coordinates,text_cleaned,origdate_text,objecttype
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HD002097,"[{'gender': 'male', 'cognomen': 'ÎÎ±Î»Î»ÎµÎ½Ï...",checked with photo,"Patrasso - AthÃ­nai, zwischen",2012-03-15,GrÃ¤f,30 cm,Greek-Latin,"CIL 03, 00572.; CIL 03, 07306.; IG 02 (2. Aufl...",146 cm,[ ]ΥΤΟΚΡΑΤΟΡΙ / [ ]ΑΙΣΑΡΙ / [[[ ]]] / [ ]ΥΣΕΒΕ...,0395,None,"Marmor, geÃ¤dert / farbig",https://www.trismegistos.org/text/177037,[Α]ὐτοκράτορι / [Κ]αίσαρι / [[[---]]] / [Ε]ὐσε...,Meilenstein mit zwei griechischen Inschriften...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,AttikÃ­,mile-/leaguestone,"AthÃ­nai, Epigr. Mus.","Athenae, bei",0397,mile-/leaguestone,2.7 cm,data available,"Dafni, byzantinisches Kloster, bei, sekundÃ¤r ...",None,None,None,None,None,None,"38.012978,23.635883",Αὐτοκράτορι Καίσαρι [] Εὐσεβεῖ Εὐτυχει Σεβαστῶ...,395 AD – 397 AD,"[Meilen-/Leugenstein, 89]"
HD008470,"[{'name': 'M. Iul. Bassi', 'person_id': '1', '...",checked with photo,PÃ¡trai,2012-04-16,Cowey,52.5 cm,Latin,"AE 1979, 0571. (B); M. Å aÅ¡el Kos, Inscriptio...",(176) cm,D D M IVL BASSI / / D D,0001,None,None,https://www.trismegistos.org/text/177068,D(---) D(---) M(arci) Iul(i) Bassi // D(---) D...,(B): Abweichende Textgestaltung der Z. 2.,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,PelopÃ³nissos,mile-/leaguestone,"PÃ¡trai, Arch. Mus.",Colonia Augusta Aroe Patrae,0200,mile-/leaguestone,6.5-4.9 cm,None,None,1974,None,None,None,None,None,"38.251123,21.741943",D D Marci Iuli Bassi D D,1 AD – 200 AD,"[Meilen-/Leugenstein, 89]"
HD012328,"[{'cognomen': 'Nerva Traianus', 'gender': 'mal...",checked with photo,EpitÃ¡lion,2010-10-29,Cowey,30 cm,Latin,"AE 1969/70, 0589.; P.G. Themelis, AEph 1969/70...",150 cm,IMP CAISAR DI / VI NERVAI F NERVA / TRAIANVS O...,0114,None,None,https://www.trismegistos.org/text/177088,Imp(erator) Caisar(!) di/vi Nervai(!) f(ilius)...,Auf dem Foto sind die Buchstaben kaum erkannbar.,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,DytikÃ­ EllÃ¡da,mile-/leaguestone,None,Epitalion,0115,mile-/leaguestone,None,None,None,1967,None,None,None,None,None,"37.63333,21.5",Imperator Caisar sic divi Nervai sic filius Ne...,114 AD – 115 AD,"[Meilen-/Leugenstein, 89]"
HD033597,"[{'person_id': '1', 'name': 'M. Aureli[us] Ant...",checked with photo,PÃ¡trai,2014-10-10,Cowey,40 cm,Latin,"AE 1899, 0015.; H. von Prott, MDAI(A) 23, 1898...",181 cm,IMP CAESA[ ] / M AVRELIV[ ] / ANTONINV[ ] / AV...,0164,None,limestone: rocks - clastic sediments,https://www.trismegistos.org/text/177185,Imp(erator) Caesa[r] / M(arcus) Aureliu[s] / A...,(B): Am Zeilenende von Z. 1-2 ein Buchstaben ...,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Achaia,PelopÃ³nissos,mile-/leaguestone,"PÃ¡trai, Arch. Mus.",Colonia Augusta Aroe Patrae,0166,mile-/leaguestone,4.5-5.5 cm,None,None,1898,None,None,None,None,None,"38.251123,21.741943",Imperator Caesar Marcus Aurelius Antoninus Aug...,164 AD – 166 AD,"[Meilen-/Leugenstein, 89]"
HD037019,None,checked with photo,Daphni,2011-10-31,Cowey,None,Greek,"AE 2001, 1832.; E. Sironen, ZPE 136, 2001, 264...",None,[ ] / ΙΟΒΙΑΝ[ ] / ΤΟΙΣ ΔΕΣΠΟ / ΤΑΙΣ ΗΜΩΝ / ΤΕΙ...,0363,None,None,https://www.trismegistos.org/text/177199,[--- καὶ] / Ἰοβιαν[ῷ] / τοῖς δεσπό/ταις ἡμῶν /...,None,https://edh-www.adw.uni-heidelberg.de/edh/geog...,Greece,https://edh-www.adw.uni-heid